In [ ]:
import ipyleaflet
import ogr, os
import pandas as pd
import geopandas as gpd
import shapefile
import json  
from shapely.geometry import Polygon
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles
)

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile




#ROI name
roi="NorthCarolina"
#Set buffer distance in decimal degreees
dist=0.01


watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

m = Map(layers=(watercolor, ), center=(35,-75 ), zoom=7.5)


dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 circlemarker={},
                 )
def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)


m

In [ ]:
bnds= (dc.last_draw).get("geometry").get("coordinates")[0]
bnds_poly=Polygon(bnds)
bounds_gpd=gpd.GeoDataFrame(geometry=[bnds_poly])
bounds_gpd=bounds_gpd.set_crs('EPSG:4326')

print("Bounds shape created from drawing")

coast=gpd.read_file('https://geodata.lib.berkeley.edu/download/file/stanford-xv279yj9196-geojson.json')

print('US coastline saved')


#clip coastal polyline
roi_coast=gpd.clip(coast, bounds_gpd)
roi_coast=roi_coast.to_crs('EPSG:4326')
buffer=roi_coast.buffer(dist)

print('US Coastline buffered to input distance and clipped to drawn bounds area')


foot_url='https://usbuildingdata.blob.core.windows.net/usbuildings-v2/NorthCarolina.geojson.zip'
footfile= urlopen(foot_url).read()
footfile=BytesIO(footfile)
foot_doc=ZipFile(footfile)
foot_cont=foot_doc.open('NorthCarolina.geojson')
foot_gpd=gpd.read_file(foot_cont)

print('MS data saved')

json_clipped = gpd.clip(foot_gpd, buffer)
json_clipped.to_file('.../ROI_Bldgs.geojson', driver='GeoJSON')

print('MS data clipped to coastal buffer roi and downloaded')